# More `pandas` Tools

<img src="images/pandas1.jpeg" style="width:500px;height:500px">



<img src="images/pandas3.jpeg" style="width:500px;height:500px">

## Learning Goals

- Use mapping tools to transform data frame columns
- Handle missing data
- Use GroupBy objects to organize and aggregate data
- <b>Coding techniques</b> 

## Workflow
<p>
We are going to develope a workflow to answer some questions using our Animal dataset from last week

1. Read Data
2. Investigate data (let's consider this the 1st time we see the data)
3. Clean data <b>if needed</b>
4. Create new Data if needed
5. Compute needed statistics
6. Rinse and Repeat 

In [1]:
import pandas as pd
import numpy as np

We'll work with the Austin Animal Center dataset and with data from [King County's Department of Assessments (Seattle housing data)](https://info.kingcounty.gov/assessor/DataDownload/default.aspx). On King County's page download two files: "Real Property Sales" and "Residential Building". Then unzip them.

In [2]:
json_url = 'https://data.austintexas.gov/resource/9t4d-g238.json'
animals = pd.read_json(json_url)

<b>Primary Question</b>:<br>
What is the average age for adopted animals?

<p>
<b>Secondary Quesiton(s):</b>
    
- In our shelter, are cats or dogs older?
- Are there more males or females in the shelter?




In [ ]:
animals.head()

In [ ]:
#animals.info()
animals.describe()

In [ ]:
animals["age_upon_outcome"].unique()
#animals["age_upon_outcome"].value_counts()

There are 4 different time units used in "age_upon_outcome"
- days
- months
- years
- weeks

<p>
How to handle NULL?

## `apply()`

- Used to apply a function to a Dataframe or Series
- Used when more complex operations need to be performed
- `.map()` and `.applymap()` are other options
- `.apply()` more versatile than `.map()`
- apply(function_name,axis=1)

In [ ]:
def setAge(row):
    age_old = row["age_upon_outcome"]
    
    if (age_old != "NULL"):
        spl = age_old.split(" ")

        if (spl[1] == "months"):
            age_new = int(spl[0])
        elif (spl[1] == "years"):
            age_new = int(spl[0])*12
        elif (spl[1] == "days"):
            age_new = int(spl[0])/30
        else:
            print("roh-roh error ",spl[0],spl[1])
            age_new = -2 
    else:
        age_new = -1
        
        
    return age_new


animals["age_months"] = animals.apply(setAge,axis=1)

In [ ]:
def setAge(row):
## Convert "age_upon_outcome" to months 
##    Years =   x12
##    Months =  x1 
##    Weeks =  /4
##    days =  /30
##  Set NULL values to NaN 
##  Set bad values to -2 
    age_old = row["age_upon_outcome"]
    
    if (age_old != "NULL"):  # process  values that are NOT NULL
##  Split the field into 2 columns... split on white spaces        
        spl = age_old.split(" ")
        if (spl[1] == "months" or spl[1] == "month"):
            age_new = int(spl[0])
        elif (spl[1] == "years" or spl[1] == "year"):
            age_new = int(spl[0])*12
        elif (spl[1] == "weeks" or spl[1] == "week"):
            age_new = int(spl[0])/4
        elif (spl[1] == "days" or spl[1] == "day"):
            age_new = int(spl[0])/30
        else:  # This covers cases we have not anticipated OR are bad values 
            print("roh-roh error ",spl[0],spl[1])
            age_new = -2 
    else:
        age_new = np.nan
        
    return age_new

animals["age_months"] = animals.apply(setAge,axis=1)

In [ ]:
animals["age_months"].value_counts()

## Mapping

### `Series.map()`

The `.map()` method applies a transformation to every entry in the Series. This transformation  "maps" each value from the Series to a new value. A transformation can be defined by a function, Series, or dictionary - usually we'll use functions.

The `.apply()` method is similar to the `.map()` method for Series, but can only use functions. It has more powerful uses when working with DataFrames.

In [ ]:
def one_year(age):
    if age == '1 year':
        age =  '1 years'
    elif age == "1 month":
        age = "1 months"
    elif age == "1 week":
        age = "1 weeks"   
    return age
    
animals['new_age1'] = animals['age_upon_outcome'].map(one_year)

In [ ]:
animals['new_age1'].value_counts()

### More Sophisticated Mapping

Let's use `.map()` to turn sex_upon_outcome into a category with three values (called **ternary**): male, female, or unknown. 

First, explore the unique values:

One of our secondary questions is "Do males live longer than females?"

In [ ]:
animals['sex_upon_outcome'].value_counts()

In [ ]:
def sex_mapper(status):
    if status in ['Neutered Male', 'Intact Male']:
        return 'Male'
    elif status in ['Spayed Female', 'Intact Female']:
        return 'Female'
    else:
        return 'Unknown'

In [ ]:
animals['new_sex1'] = animals['sex_upon_outcome'].map(sex_mapper)
animals.loc[:, ['sex_upon_outcome', 'new_sex1']]

### Lambda Functions

Simple functions can be defined just when you need them, when you would call the function. 
- These are called **lambda functions**. These functions are **anonymous** and disappear immediately after use.
- The functions help preserve resources

Let's use a lambda function to get rid of 'Other' in the "animal_type' column.

In [ ]:
animals[animals['animal_type'] == 'Other']

In [ ]:
animals['animal_type'].value_counts()

In [ ]:
animals['animal_type'].map(lambda x: np.nan if x == 'Other' else x).value_counts()

We haven't modified the actual data frame!

In [ ]:
print(animals['animal_type'].value_counts())


## Handling Missing Data

A lot of the times we'll have missing information in our data set. This can sometimes be troublesome in what we're trying to do.

So far, we've been doing some preprocessing/cleaning to answer questions. Now we're going to handle the missing values in our data.

There are a few strategies we can choose from and they each have their special use case.

> Before making changes, it's convenient to make changes to a copy instead of overwriting data. We'll keep all our changes in `animals_clean` which will be a [copy](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.copy.html) of the original DataFrame.

In [ ]:
animals_clean = animals.copy()

### Fill with a Relevant Value

A lot of times we already have an idea of how we want to specify that a value was missing and replace it with a value that makes more sense than an "empty" value.

For example, it might make sense to fill the value as "MISSING" or "UNKNOWN". This way it's clearer when do more analysis.

> We can use Pandas' [`fillna()` method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html) to replace missing values with something specific

In [ ]:
animals.info()
#animals[animals['name'].isna()].head()

In [ ]:
#animals_name_filled = animals.fillna({'name':'UNKNOWN'}) 
animals_only_names = animals[['name']].fillna(value='UNKNOWN')
print(animals_only_names)

In [ ]:
# To keep changes in DataFrame, overwrite the column
animals_clean[['name']] = animals_only_names
animals_clean[animals_clean['name'].isna()].head()

### Fill with a Reasonable Value

Other times we don't know what the missing value was but we might have a reasonable guess. This allows us to still use the data point (row) in our analysis.

> Beware that filling in missing values can lead to you drawing incorrect conclusions. If most of the data from a column are missing, it's going to appear that the value you filled it in with is more common that it actually was!

A lot of the time we'll use the _mean_ or _median_ for numerical values. Sometimes values like $0$ make sense since it might make sense in the context of how the data was collected.

With categorical values, you might choose to fill the missing values with the most common value (the *mode*).

> Similar to the previous subsection, we can use the `fillna()` method after specifying the value to fill

In [ ]:
# Let's find the most common value for `outcome_subtype`
outcome_subtype_counts = animals['outcome_subtype'].value_counts()
outcome_subtype_counts

In [ ]:
# This gets us just the values in order of most frequent to least frequent
outcome_subtype_ordered = outcome_subtype_counts.index
print(outcome_subtype_ordered)

In [ ]:
# Get the first one
most_common_outcome_subtype = outcome_subtype_ordered[0]

most_common_outcome_subtype

In [ ]:
# Using the built-in mode() method
# Note this is Series so we have to get the first element (which is the value)
most_common_outcome_subtype = animals['outcome_subtype'].mode()[0]
most_common_outcome_subtype

In [ ]:
# Similar to the previous subsection, we can use fillna() and update the DF
animals_clean['outcome_subtype'] = animals['outcome_subtype']\
.fillna(most_common_outcome_subtype)
animals_clean.tail()
print(animals_clean['outcome_subtype'].value_counts())
print(animals['outcome_subtype'].value_counts())

### Drop Missing Data

You should try to keep as much relevant data as possible, but sometimes the other methods don't make as much sense and it's better to remove or **drop** the missing data.

We typically drop missing data if very little data would be lost and/or trying to fill in the values wouldn't make sense for our use case. For example, if you're trying to predict the outcome based on the other features/columns it might not make sense to fill in those missing values with something you can't confirm.

> We noticed that `outcome_type` had only a few missing values. It might not be worth trying to handle those few missing values. We can pretend that the `outcome_type` was an important feature and without it the rest of the row's data is of little importance to us.
>
> So we'll decide to drop the row if a value from `outcome_type` is missing. We'll use Pandas' [`dropna()` method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html).

In [ ]:
# This will drop any row (axis=0) or column (axis=1) that has missing values
animals_clean = animals_clean.dropna(   # Note we're overwriting animals_clean
                                axis=0, # This is the default & will drop rows;
                                        # axis=1 for cols
                                subset=['outcome_type'] # Specific labels
                                                        # to consider (defaults to "all")
)
animals_clean.info()

### Comparing Before and After

We can now see all the work we did!

In [ ]:
# Original data
animals.info()

In [ ]:
# Missing data cleaned
animals_clean.info()

## Analysis Time 
- Cleaning is now Done

## Aggregating over DataFrames: `.groupby()`

Those of you familiar with SQL have probably used the GROUP BY command. (And if you haven't, you'll see it very soon!) Pandas has this, too.

The `.groupby()` method is especially useful for aggregate functions applied to the data grouped in particular ways.

In [ ]:
animals_clean.groupby('animal_type')

We can group by multiple columns, and also return a DataFrameGroupBy object

Notice the object type [DataFrameGroupBy](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html) object. 

In [ ]:
animals_clean.columns

### `.groups` and `.get_group()`

In [ ]:
# This retuns each group indexed by the group name:
# e.g. 'Bird', along with the row indices of each value
animals_clean.groupby('animal_type').groups

Once we know we are working with a type of object, it opens up a suite of attributes and methods. One attribute we can look at is groups.

In [ ]:
# Once we know the group indices, we can return the groups using those indices.
animals_clean.groupby('animal_type').get_group('Dog')

#### Multi-Indexing

In [ ]:
# Same goes for multi index groupbys
animal_outcome = animals_clean.groupby(['animal_type', 'outcome_type'])
animal_outcome.groups

In [ ]:
# animal_outcome.groups is a dictionary, so we can access the group names using keys()
animal_outcome.groups.keys()

In [ ]:
# We can then get a specific group, such as Cats that were adopted
animal_outcome.get_group(('Cat', 'Adoption'))

### Aggregating

Once again, as we will see in SQL, groupby objects are intended to be used with aggregation. In SQL, we will see that our queries that include GROUP BY require aggregation performed on columns.

We can use `.sum()`, `.mean()`, `.count()`, `.max()`, `.min()`, etc. Find a list of common aggregations [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html).

In [ ]:
animals_clean.groupby('animal_type').mean()
ages_mean = 

In [ ]:
animals_clean.groupby('animal_type').mean()
ages_mean = animals_clean.groupby('animal_type').mean()
#ages_mean = animals_clean.groupby('animal_type',as_index=False).mean()
#print(ages_mean[ages_mean["animal_type"] == "Cat"]["age_months"])

In [ ]:
animals_clean.groupby('animal_type').max()
#animals_clean.groupby('animal_type').min()

 You can perform multiple groupby operations at the same times use the agg() function

In [ ]:
animals.groupby(by=['outcome_type']).agg({"age_months":["mean","max","min"]})

## Level Up: Pivoting a DataFrame

### `.pivot_table()`

Those of you familiar with Excel have probably used Pivot Tables. Pandas has a similar functionality.

Grouping by two different columns can be very helpful.

But it has the unsavory side effect of creating a two-level index. This can be a good time to use `.pivot_table()`.

(There is also a `.pivot()`. For the somewhat subtle differences, see [here](https://stackoverflow.com/questions/30960338/pandas-difference-between-pivot-and-pivot-table-why-is-only-pivot-table-workin).)

#### Example

In [ ]:
df = pd.DataFrame({"sex": ["male", "male", "male", "male", "male",
                          "female", "female", "female", "female"],
                    "num_puppies": ["one", "one", "one", "two", "two",
                          "one", "one", "two", "two"],
                    "breed": ["terrier", "retriever", "retriever", "terrier",
                          "terrier", "retriever", "terrier", "terrier",
                          "retriever"],
                    "past_owners": [1, 2, 2, 3, 3, 4, 5, 6, 7],
                    "family_members": [2, 4, 5, 5, 6, 6, 8, 9, 9]})
df

In [ ]:
# This first example aggregates values by taking the sum.

table = pd.pivot_table(df, values='past_owners', index=['sex', 'num_puppies'],
                     columns=['breed'], aggfunc=np.sum)
table

### Exercise

Use `.pivot_table()` to add up the number of my tasks by category. Hint: Use `sum()` as your aggregating function.

In [ ]:
tasks = pd.DataFrame({'category': ['house', 'house', 'school', 'school'],
                      'descr': ['kitchen', 'laundry', 'git', 'Python'],
                      'priority': [2, 3, 4, 1], 'num_tasks': [2, 1, 2, 3]})

tasks

<details>
    <summary>Answer</summary>
    <code>tasks.pivot_table(values='num_tasks', index='category', aggfunc=sum)</code>
    </details>

## Level Up: Methods for Combining DataFrames: `.join()`, `.merge()`, `pd.concat()`

### `.join()`

In [ ]:
toy1 = pd.DataFrame([[63, 142], [33, 47]], columns=['age', 'HP'])
toy2 = pd.DataFrame([[63, 100], [33, 200]], columns=['age', 'MP'])

In [ ]:
toy1

In [ ]:
toy2

In [ ]:
# We can't just join these as they are, since we haven't specified our suffixes.

toy1.join(toy2)

In [ ]:
toy1.join(toy2, lsuffix='1', rsuffix='2')

If we don't want to keep both, we could set the overlapping column as the index in each DataFrame:

In [ ]:
toy1.set_index('age').join(toy2.set_index('age'))

For more on this method, check out the [doc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html)!

In [ ]:
### `.merge()`

Or we could use `.merge()`:

In [ ]:
toy1.merge(toy2)

In [ ]:
ds_chars = pd.read_csv('data/ds_chars.csv', index_col=0)
ds_chars

In [ ]:
states = pd.read_csv('data/states.csv', index_col=0)
states

### The `how` Parameter

This parameter in both `.join()` and `.merge()` tells the compiler what sort of join to effect.

In [ ]:
ds_chars.merge(states,
               left_on='home_state',
               right_on='state',
               how='inner')

In [ ]:
ds_chars.merge(states,
               left_on='home_state',
               right_on='state',
               how='outer')

### `pd.concat()`

This method takes a *list* of pandas objects as arguments.

In [ ]:
ds_full = pd.concat([ds_chars, states])
ds_full

`pd.concat()`–– and many other pandas operations –– make use of an `axis` parameter. For this particular method I need to specify whether I want to concatenate the DataFrames *row-wise* (`axis=0`) or *column-wise* (`axis=1`). The default is `axis=0`, so let's override that!

In [ ]:
ds_full = pd.concat([ds_chars, states], axis=1)
ds_full

## King County Assessments

As data scientists, we want to build a model to predict the sale price of a house in Seattle in 2019, based on its square footage. We know that the King County Department of Assessments has comprehensive data available on real property sales in the Seattle area. We need to prepare the data.

In [ ]:
# You'll need to use a new encoding here. List of all encodings here:
# https://docs.python.org/3/library/codecs.html#standard-encodings

# Both of these csv files have many columns, so we'll just pre-select
# which ones we want to use.

sales_df = pd.read_csv('/Users/gdamico/Downloads/EXTR_RPSale.csv',
                       encoding='latin-1',
                       usecols=['Major', 'Minor', 'DocumentDate', 'SalePrice'])

In [ ]:
sales_df.info()

In [ ]:
bldg_df = pd.read_csv('~/Downloads/EXTR_ResBldg.csv',
                     usecols=['Major', 'Minor', 'SqFtTotLiving', 'ZipCode'])

In [ ]:
bldg_df.info()

In [ ]:
# See the Level Up sections for more on merging!
sales_data = pd.merge(sales_df, bldg_df, on=['Major', 'Minor'])

In [ ]:
sales_data.head()

In [ ]:
sales_data.info()

We can see right away that we're missing ZIP codes for many of the sales transactions.

In [ ]:
sales_data[sales_data['ZipCode'].isna()].head()

## Exercises
Important: Do these *in order*!

1. What percentage of housing records are missing ZIP codes?

<details>
    <summary>Answer</summary>
    <code>sales_data['ZipCode'].isna().sum() / sales_data.shape[0]</code>
    </details>

Let's drop the rows with missing zip codes.

In [ ]:
sales_data = sales_data.loc[~sales_data['ZipCode'].isna(), :]

sales_data.head()

2. Investigate and drop rows with invalid values in the SalePrice and SqFtTotLiving columns.

<details>
    <summary>One possible answer here</summary>
    <code>sales_data = sales_data[sales_data['SalePrice'] > 10000]</code>
    </details>

3. Investigate and handle non-numeric ZipCode values

Can you find a way to shorten ZIP+4 codes to the first five digits?

In [ ]:
def is_integer(x):
    try:
        _ = int(x)
    except ValueError:
        return False
    return True

sales_data.loc[sales_data['ZipCode'].apply(is_integer) == False, 'ZipCode'].head()

<details>
    <summary>One possible answer here</summary>
    <code>def five_digit_ZIP(x):
    try:
        return int(str(x)[:5])
    except:
        return x
sales_data['ZipCode'] = sales_data['ZipCode'].map(five_digit_ZIP)
sales_data = sales_data.loc[sales_data['ZipCode'].apply(is_integer) == True, :]
sales_data['ZipCode'] = sales_data['ZipCode'].map(int)</code>
    </details>

4. Add a column for PricePerSqFt.

<details>
    <summary>Answer here</summary>
    <code>sales_data['PricePerSqFt'] = sales_data['SalePrice'] / sales_data['SqFtTotLiving']</code>
    </details>

5. Subset the data to 2021 sales only.

We can assume that the DocumentDate is approximately the sale date. The first thing you should do is convert the date to a datetime object with the following code:

<code>sales_data['DocumentDate'] = pd.to_datetime(sales_data['DocumentDate'])</code>

<details>
    <summary>Answer here</summary>
    <code>sales_data['DocumentDate'] = pd.to_datetime(sales_data['DocumentDate'])
sales_data = sales_data.loc[sales_data['DocumentDate'] > '12/31/2020']</code>
    </details>

6. What is the mean price per square foot for a house sold in Seattle in 2021?

<details>
    <summary>Answer here</summary>
    <code>sales_data['PricePerSqFt'].mean()</code>
    </details>

7. I'm interested in seeing all the Major ID nos. that cover multiple ZIP codes. The first step will be to:

- group by 'Major' and then check out the ZIP codes for each 'Major' value. Let's store that in a new variable called "grouped".

<details>
    <summary>Answer here</summary>
    <code>grouped = sales_data.groupby('Major')['ZipCode'].value_counts()</code>
    </details>

In [ ]:
grouped

8. Then I'll need to:

- iterate over that Series, looking for entries where the first half of the index (the 'Major' ID) corresponds to two different "second halves" of the index (the 'ZipCode'). One way to proceed would be to initialize an empty list, and then add elements of the index to it when the 'Major' value of an entry matches the 'Major' value of the next entry.

<details>
    <summary>Answer here</summary>
<code>multiples = []
for j in range(len(grouped) - 1):
    if grouped.index[j+1][0] == grouped.index[j][0]:
        multiples.append(grouped.index[j])
        multiples.append(grouped.index[j+1])</code>

9. Which ZIP Code has had the most sales in 2021, and how many has it had?

<details>
    <summary>Answer here</summary>
    <code>sales_data.groupby('ZipCode').count().sort_values(by='Major', ascending=False).head(1)</code>
    </details>

10. Looking ahead: Plotting!

a. What happens if we run:

<code>sales_data['SqFtTotLiving'].hist();</code>?

b. What about:

<code>sales_data_sorted = sales_data.sort_values('SqFtTotLiving')
sales_data_sorted.plot(x='SqFtTotLiving', y='SalePrice');</code>?

c. How could we plot the number of sales by date?

<details>
    <summary>Answer here</summary>
    <code>ctr = sales_data.groupby('DocumentDate').count().reset_index()
ctr.plot(x='DocumentDate', y='SalePrice') # _Any_ column will work here for the  y-value!;</code>
    </details>